In [1]:
from classifiers import ConformalPredictor, WCRF, vanillaRF
from data import load_dataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from eval import conformal_pred_eval, wcrf_eval, evaluate_model
import numpy as np


In [2]:
DATASETS = ["german_credit_data", "heart_disease_data", "breast_cancer_data", "compas_data"]
CLASSIFIERS = ['vanillaRF', 'WCRF', 'ConformalPredictor']

## Load pre-processed data

In [3]:
datasets = {}
for dataset_name in DATASETS:
    print(f"Processing dataset: {dataset_name}")
    data = load_dataset(dataset_name)
    datasets[dataset_name] = data

Processing dataset: german_credit_data
Processing dataset: heart_disease_data
Processing dataset: breast_cancer_data
Processing dataset: compas_data
Index(['priors_count', 'age', 'juv_fel_count', 'juv_misd_count',
       'juv_other_count', 'above_45', 'below_25', 'is_felon', 'is_misdemeanor',
       'Female', 'Male', 'African-American', 'Asian', 'Caucasian', 'Hispanic',
       'Native American', 'Other', 'Cluster0', 'Cluster1', 'Cluster2',
       'Cluster3'],
      dtype='object')


## Initialize models
TODO - hyper-parameter tuning

In [4]:
classifiers = {}
for dataset_name in DATASETS:
    X_train, X_calib, X_test, y_train, y_calib, y_test = datasets[dataset_name]["train"], datasets[dataset_name]["calib"], datasets[dataset_name]["test"], datasets[dataset_name]["train_labels"],datasets[dataset_name]["calib_labels"],datasets[dataset_name]["test_labels"]
    rfc = vanillaRF(X_train, y_train)
    # Conformal RF
    cpc = ConformalPredictor()
    # Weighted Cautiouse Random Forest
    wcrf = WCRF(s=1,gamma=1,labda=5)
    classifiers[str(dataset_name)+'__vanillaRF'] = rfc
    classifiers[str(dataset_name)+'__ConformalPredictor'] = cpc
    classifiers[str(dataset_name)+'__WCRF'] = wcrf



## Fit models

In [5]:
for classifier in classifiers.keys():
    # Vanilla Random Forest
    dataset_name = classifier.split('__')[0]
    if "vanillaRF" in str(classifier): 
        best_params = classifiers[classifier].gridSearch()
        classifiers[classifier].fit(best_params)
    # Conformal RF
    elif "ConformalPredictor" in str(classifier):
        classifiers[classifier].fit(datasets[dataset_name]['train'], datasets[dataset_name]['train_labels'], datasets[dataset_name]['calib'], datasets[dataset_name]['calib_labels'])
    # Weighted Cautiouse Random Forest
    elif "WCRF" in str(classifier):
        classifiers[classifier].fit(datasets[dataset_name]['train'], datasets[dataset_name]['train_labels'])
        classifiers[classifier].fit_w(datasets[dataset_name]['train'], datasets[dataset_name]['train_labels'])
    else: 
        raise ValueError(f"Classifier: {classifier} not found")

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_

## Run Eval

In [6]:
for classifier in classifiers.keys():
    classifier_name = classifier.split("__")[1]
    dataset_name = classifier.split("__")[0]
    print(f'Eval for: {classifier_name} and {dataset_name}')
    model_eval = evaluate_model(classifier_name, datasets[dataset_name]['test'], datasets[dataset_name]['test_labels'], classifiers[classifier])
    print(model_eval)

# Vanilla Random Forest
print(accuracy_score(y_test, rfc.predict(X_test)))
# Conformal RF
print(conformal_pred_eval(X_test,y_test,cpc))
# Weighted Cautiouse Random Forest
print(wcrf_eval(X_test,y_test, wcrf))



Eval for: vanillaRF and german_credit_data
0.74
Eval for: ConformalPredictor and german_credit_data
{'u65_score': 71.72, 'single_set_accuracy': 85.07, 'determinacy': 33.5}
Eval for: WCRF and german_credit_data
{'u65_score': 76.0, 'single_set_accuracy': 76.0, 'determinacy': 100.0, 'precise_accuracy': 76.5}
Eval for: vanillaRF and heart_disease_data
0.8618421052631579
Eval for: ConformalPredictor and heart_disease_data
{'u65_score': 88.46, 'single_set_accuracy': 97.71, 'determinacy': 71.71}
Eval for: WCRF and heart_disease_data
{'u65_score': 86.18, 'single_set_accuracy': 86.18, 'determinacy': 100.0, 'precise_accuracy': 86.18}
Eval for: vanillaRF and breast_cancer_data
0.9574468085106383
Eval for: ConformalPredictor and breast_cancer_data
{'u65_score': 75.13, 'single_set_accuracy': 95.24, 'determinacy': 33.51}
Eval for: WCRF and breast_cancer_data
{'u65_score': 96.81, 'single_set_accuracy': 96.81, 'determinacy': 100.0, 'precise_accuracy': 96.81}
Eval for: vanillaRF and compas_data
0.66388